In [ ]:
# 건강보험심사평가원 제공 Open API 사용 병원 현황 데이터 조회
# 진료과목코드 선택해서 해당 진료과목을 지원하는 병원 조회
# 테이블 만들기 - 테이블에 조회된 데이터 저장
# 자치구 또는 시도별 병원 현황 조회해서 파일로 저장

In [12]:
import urllib
import requests

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xml.etree import ElementTree as ET
import json

import pymysql
import time

In [2]:
api_key = "4GAO8lWy1e5z9d7XJpsl6SmHTBzGefgEJZIW9UqrHaIoYq4TW5fHpqCqzHhln4PLQnFns/UQmvTHbLd1R/kOmg=="

url = "http://apis.data.go.kr/B551182/hospInfoServicev2/getHospBasisList"

params ={
    'ServiceKey' : api_key,
    'dgsbjtCd' : '07',      # 심장 혈관 흉부외과
    'pageNo' : '1',
    'numOfRows' : '2000'
    }

resp = requests.get(url, params = params)

print(resp.content.decode('utf-8'))

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><addr>경기도 수원시 팔달구 중부대로 93, (지동)</addr><clCd>01</clCd><clCdNm>상급종합</clCdNm><cmdcGdrCnt>0</cmdcGdrCnt><cmdcIntnCnt>0</cmdcIntnCnt><cmdcResdntCnt>0</cmdcResdntCnt><cmdcSdrCnt>0</cmdcSdrCnt><detyGdrCnt>0</detyGdrCnt><detyIntnCnt>0</detyIntnCnt><detyResdntCnt>0</detyResdntCnt><detySdrCnt>7</detySdrCnt><drTotCnt>260</drTotCnt><emdongNm>지동</emdongNm><estbDd>19670523</estbDd><hospUrl>http://www.cmcvincent.or.kr/skip.html</hospUrl><mdeptGdrCnt>1</mdeptGdrCnt><mdeptIntnCnt>1</mdeptIntnCnt><mdeptResdntCnt>9</mdeptResdntCnt><mdeptSdrCnt>242</mdeptSdrCnt><pnursCnt>0</pnursCnt><postNo>16247</postNo><sgguCd>310603</sgguCd><sgguCdNm>수원팔달구</sgguCdNm><sidoCd>310000</sidoCd><sidoCdNm>경기</sidoCdNm><telno>031-1577-8588</telno><XPos>127.0274271</XPos><YPos>37.2779855</YPos><yadmNm>가톨릭대학교 성빈센트병원</yadmNm><ykiho>JDQ4MTYyMiM1MSMkMSMkMCMkODkkMzgxMzU

In [ ]:
result = ET.fromstring(resp.content.decode('utf-8'))    # xml string -> obiect tree 변환

In [ ]:
# xml을 읽어서 만든 객체 트리구조 확인 
print(result.tag)

for c1 in result:               # result 태그 하위 요소를 순회
    print(" " * 2, c1.tag)
    for c2 in c1:               # c1 태그 하위 요소를 순회
        print(" " * 4, c2.tag)
        for c3 in c2:           # c2 태그 하위 요소를 순회
            print(" " * 6, c3.tag)

response
   header
     resultCode
     resultMsg
   body
     items
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       item
       

In [13]:
# 실제 데이터가 있는 item 태그를 읽어 데이터 뽑기
items = result.find('body').find('items').findall('item')   # find는 1개, findall는 전체 조회

columns = ['yadmNm', 'clCdNm', 'sidoCdNm', 'sgguCdNm','addr', 'emdongNm', 'hospUrl']
data = []
for item in items:
    row = {}
    for c in item:
        if c.tag in columns:
            row.update({c.tag:c.text})
    data.append(row)

In [14]:
df_hosp = pd.DataFrame(data)
df_hosp.head(3)

,addr,clCdNm,emdongNm,hospUrl,sgguCdNm,sidoCdNm,yadmNm
0,"경기도 수원시 팔달구 중부대로 93, (지동)",상급종합,지동,http://www.cmcvincent.or.kr/skip.html,수원팔달구,경기,가톨릭대학교 성빈센트병원
1,"인천광역시 부평구 동수로 56, (부평동)",상급종합,부평동,http://www.cmcism.or.kr/,인천부평구,인천,가톨릭대학교인천성모병원
2,"강원특별자치도 강릉시 사천면 방동길 38, (사천면)",상급종합,사천면,http://www.gnah.co.kr,강릉시,강원,강릉아산병원


In [16]:
# 데이터 조회 2 . 전체 데이터 조회

api_key = "4GAO8lWy1e5z9d7XJpsl6SmHTBzGefgEJZIW9UqrHaIoYq4TW5fHpqCqzHhln4PLQnFns/UQmvTHbLd1R/kOmg=="
url = "http://apis.data.go.kr/B551182/hospInfoServicev2/getHospBasisList"


columns = ['yadmNm', 'clCdNm', 'sidoCdNm', 'sgguCdNm','addr', 'emdongNm', 'hospUrl']
cnt_per_page = 100  # 한번 요청할때 받아오는 데이터 개수
page = 1
data = []
while True:    
    params = {
        "ServiceKey": api_key,
        "dgsbjtCd": "07", # 심장혈관흉부외과
        "pageNo": page, 
        "numOfRows": cnt_per_page
    }
    resp = requests.get(url, params)
    parsed_xml = ET.fromstring(resp.content.decode('utf-8'))
    items = parsed_xml.find('body').find('items').findall("item")
    for item in items:
        row = {}
        for child in item:
            if child.tag in columns:
                row.update({child.tag: child.text})
        data.append(row)
    total_cnt = int(parsed_xml.find('body').find('totalCount').text)    # 전체 데이터 개수 읽기
    if total_cnt <= len(data):
        break

    page += 1

    time.sleep(0.1) # 여기서 0.1초 실행 중지

In [ ]:
print(len(data))

df_all_hospitals = pd.DataFrame(data)
print(df_all_hospitals.shape)
df_all_hospitals.info()     # missing value 확인 -> 테이블 만들때 컬럼을 null 허용 처리


1137
(1137, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137 entries, 0 to 1136
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   addr      1137 non-null   object
 1   clCdNm    1137 non-null   object
 2   emdongNm  1128 non-null   object
 3   hospUrl   282 non-null    object
 4   sgguCdNm  1137 non-null   object
 5   sidoCdNm  1137 non-null   object
 6   yadmNm    1137 non-null   object
dtypes: object(7)
memory usage: 62.3+ KB


In [ ]:
df_all_hospitals.fillna('',inplace=True)    # nan ->'' 변경 (원본 변경 : inplace=True )

In [ ]:
df_all_hospitals.info()                     # nan 처리 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1137 entries, 0 to 1136
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   addr      1137 non-null   object
 1   clCdNm    1137 non-null   object
 2   emdongNm  1137 non-null   object
 3   hospUrl   1137 non-null   object
 4   sgguCdNm  1137 non-null   object
 5   sidoCdNm  1137 non-null   object
 6   yadmNm    1137 non-null   object
dtypes: object(7)
memory usage: 62.3+ KB


In [24]:
try:
    conn = pymysql.connect(
        host= "127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )
    cursor = conn.cursor()


    sql = "drop table if exists hospital2"
    cursor.execute(sql)
    sql = """ create table if not exists hospital2
         ( 
            idx int primary key auto_increment,
            addr varchar(300) not null,
            clCdNm varchar(50) not null,
            emdongNm varchar(50) null,
            hospUrl varchar(200) null,
            sgguCdNm varchar(50) not null,
            sidoCdNm varchar(50) not null,
            yadmNm varchar(50) not null
         )"""
    cursor.execute(sql)
     
except Exception as e:
    print("오류발생 : ", e)
    if conn:
        conn.rollback() 
finally:

    if cursor:
        cursor.close()
    if conn:    
        conn.close()

In [28]:
try:
    conn = pymysql.connect(
        host= "127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )

    cursor = conn.cursor()

    # 명령 실행
    sql = "delete from hospital2"
    cursor.execute(sql)

    sql = """insert into hospital2 
             (addr, clCdNm, emdongNm, hospUrl, sgguCdNm, sidoCdNm, yadmNm)
             values 
             (%s, %s, %s, %s, %s, %s, %s)"""
    data = df_all_hospitals.values.tolist()
    cursor.executemany(sql, data)

    conn.commit()

except Exception as e:
    print("오류발생 : ", e)
    if conn:
        conn.rollback()

finally:
    if cursor:
        cursor.close()
    if conn:    
        conn.close()

In [29]:
# 시도별 병원 개수 데이터를 조회해서 파일로 저장
try:
    conn = pymysql.connect(
        host= "127.0.0.1",
        user="humanda5",
        password="humanda5",
        database="data_repo"
    )
    cursor = conn.cursor()

    sql = """select sidoCdNm, count(*) cnt from hospital2 group by sidoCdNm order by cnt DESC"""
    cursor.execute(sql)

    hospital2_data = cursor.fetchall()

    df_hospital2 = pd.DataFrame(hospital2_data,  columns=['자치구', '심장혈관흉부외과병원수'] )
    df_hospital2.to_csv('data-files/hospital-heart-count-by-gu.csv', index=False)

except Exception as e:
    print("오류발생 : ", e)
finally:
    if cursor:
        cursor.close()
    if conn:    
        conn.close()